In [11]:
import glob
import mne
import pandas as pd
import numpy as np
import math
import matplotlib
#from statsmodels.stats.anova import AnovaRM
#import statsmodels.api as sm
#import seaborn as sns
#import matplotlib.pyplot as plt
#from sklearn.linear_model import LinearRegression

In [10]:
files_eeg= glob.glob(r"C:\Users\yehuda\pain_of_payment\Data_PoP\mindMonitor_*_b_*.csv", recursive= False)
files_eeg_q = glob.glob(r"C:\Users\yehuda\pain_of_payment\Data_PoP\mindMonitor_*_q_*.csv", recursive= False)

In [7]:
file=files_eeg[0]

In [9]:
ch_names = ['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']
ch_types = ['misc']+['eeg']*2+['misc']
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
#info.set_montage('standard_1020') #set the electrodes' locations
data = pd.read_csv(file) #BEN make sure you need to transpose
data['TimeStamp'] = pd.to_datetime(data['TimeStamp']).map(pd.Timestamp.timestamp)
data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
data = data[['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']]
tob = data['ToB'][0] - data['TimeStamp'][0]
t_after = tob+5
t_before = tob-5
data.to_csv("data.csv")
data = data.transpose()
raw_before = mne.io.RawArray(data, info)
raw_before.crop(tmin = t_before, tmax = tob)
raw_after = mne.io.RawArray(data, info)
raw_after.crop(tmin = tob, tmax = t_after)
raw_before.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR.
psd_before = raw_before.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 0, fmax = 4)
psd_df_before = psd_before.to_data_frame()
psd_df_before


Creating RawArray with float64 data, n_channels=4, n_times=9855
    Range : 0 ... 9854 =      0.000 ...    38.492 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=9855
    Range : 0 ... 9854 =      0.000 ...    38.492 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 100.00 Hz: -6.02, -6.02 dB

Effective window size : 1.000 (s)


C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\383722021.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
C:\Users\yehuda\AppData\Local\anaconda3\envs\my_mne_1.2_env\lib\site-packages\mne\time_frequency\psd.py:56: RuntimeWarning: Mean of empty slice
  spect = np.nanmean(spect, axis=-1)


freq  RAW_AF7  RAW_AF8
0   0.0      NaN      NaN
1   1.0      NaN      NaN
2   2.0      NaN      NaN
3   3.0      NaN      NaN
4   4.0      NaN      NaN

In [39]:
'''
may- test
'''
file  = files_eeg[0]
ch_names = ['TimeStamp', 'RAW_AF7', 'RAW_AF8']
ch_types = ['misc']+['eeg']*2
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
#info.set_montage('standard_1020') #set the electrodes' locations
PSDs_df_before_delta = pd.DataFrame()
PSDs_df_after_delta = pd.DataFrame()

data = pd.read_csv(file) #BEN make sure you need to transpose
data['TimeStamp'] = pd.to_datetime(data['TimeStamp']).map(pd.Timestamp.timestamp)
data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
data = data[['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']]
tob = data['ToB'][0] - data['TimeStamp'][0]
t_after = tob+5
t_before = tob-5

print(data)
data.drop("ToB", inplace= True, axis=1)
data.dropna(inplace=True)
data = data.transpose()

raw_before = mne.io.RawArray(data, info)
raw_before = raw_before.crop(tmin = t_before, tmax = tob)
raw_after = mne.io.RawArray(data, info)
raw_after.crop(tmin = tob, tmax = t_after)
raw_total = mne.io.RawArray(data, info)
raw_total = raw_total.crop(tmin = t_before, tmax = t_after)

raw_before.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
raw_before.notch_filter(freqs=50, picks = 'eeg')
#raw_before.plot()
raw_after.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
raw_after.notch_filter(freqs=50, picks = 'eeg')
#raw_after.plot()
raw_total.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
raw_total.notch_filter(freqs=50, picks = 'eeg')
raw_total.plot()

psd_before = raw_before.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 0, fmax = 4)
psd_df_before = psd_before.to_data_frame()
psd_after = raw_after.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 0, fmax = 4)
psd_df_after = psd_after.to_data_frame()
psd_total = raw_total.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 0, fmax = 4)
psd_df_total = psd_total.to_data_frame()
#avergaed_df = psd_df[["freq","RAW_AF7","RAW_AF8"]].mean()
#avergaed_df=avergaed_df.reset_index().T
#avergaed_df.columns = avergaed_df.iloc[0]
#avergaed_df.drop(index=avergaed_df.index[0], axis=0, inplace=True)
psd_df_before['subject'] = file[-6:-3]
psd_df_after['subject'] = file[-6:-3]

psd_df_after.to_csv(r"C:\Users\yehuda\pain_of_payment\delta_after.csv")
psd_df_before.to_csv(r"C:\Users\yehuda\pain_of_payment\delta_before.csv")
psd_df_total.to_csv(r"C:\Users\yehuda\pain_of_payment\delta_total.csv")

         TimeStamp     RAW_AF7     RAW_AF8           ToB
0     1.680290e+09         NaN         NaN  1680289811.0
1     1.680290e+09  805.860806  812.710623           NaN
2     1.680290e+09  800.219780  807.472527           NaN
3     1.680290e+09  790.549451  807.875458           NaN
4     1.680290e+09  790.146520  811.098901           NaN
...            ...         ...         ...           ...
9850  1.680290e+09  815.934066  789.743590           NaN
9851  1.680290e+09  811.098901  792.564103           NaN
9852  1.680290e+09  816.336996  790.952381           NaN
9853  1.680290e+09  820.366300  788.534798           NaN
9854  1.680290e+09  813.919414  795.384615           NaN

[9855 rows x 4 columns]
Creating RawArray with float64 data, n_channels=3, n_times=9780
    Range : 0 ... 9779 =      0.000 ...    38.199 secs
Ready.
Creating RawArray with float64 data, n_channels=3, n_times=9780
    Range : 0 ... 9779 =      0.000 ...    38.199 secs
Ready.
Creating RawArray with float64 data, n_

C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\858834473.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\858834473.py:34: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_before.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\858834473.py:37: Run

Using qt as 2D backend.
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)


In [17]:
ch_names = ['RAW_AF7', 'RAW_AF8', 'ToB', 'TimeStamp']
ch_types = ['misc']+['eeg']*2+['misc']
print(ch_types)
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
#info.set_montage('standard_1020') #set the electrodes' locations
PSDs_df_before_delta = pd.DataFrame()
PSDs_df_after_delta = pd.DataFrame()
for file in files_eeg:
    print(file)
    data = pd.read_csv(file) #BEN make sure you need to transpose
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp']).map(pd.Timestamp.timestamp)
    data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
    data = data[['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']]
    tob = data['ToB'][0] - data['TimeStamp'][0]
    t_after = tob+5
    t_before = tob-5
    data = data.transpose()
    raw_before = mne.io.RawArray(data, info)
    raw_before.crop(tmin = t_before, tmax = tob)
    raw_after = mne.io.RawArray(data, info)
    raw_after.crop(tmin = tob, tmax = t_after)
    raw_before.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_before.notch_filter(freqs=50, picks = 'eeg')
    #raw_before.plot()
    raw_after.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_after.notch_filter(freqs=50, picks = 'eeg')
    #raw_after.plot()
    psd_before = raw_before.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 0, fmax = 4)
    psd_df_before = psd_before.to_data_frame()
    psd_after = raw_after.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 0, fmax = 4)
    psd_df_after = psd_after.to_data_frame()
    #avergaed_df = psd_df[["freq","RAW_AF7","RAW_AF8"]].mean()
    #avergaed_df=avergaed_df.reset_index().T
    #avergaed_df.columns = avergaed_df.iloc[0]
    #avergaed_df.drop(index=avergaed_df.index[0], axis=0, inplace=True)
    psd_df_before['subject'] = file[-6:-3]
    psd_df_after['subject'] = file[-6:-3]
    PSDs_df_before_delta = PSDs_df_before_delta.append(psd_df_before, ignore_index = True)
    PSDs_df_after_delta = PSDs_df_after_delta.append(psd_df_after, ignore_index = True)

    PSDs_df_after_delta.to_csv(r"C:\Users\yehuda\pain_of_payment\delta_after.csv")
    PSDs_df_before_delta.to_csv(r"C:\Users\yehuda\pain_of_payment\delta_before.csv")

['misc', 'eeg', 'eeg', 'misc']
C:\Users\yehuda\pain_of_payment\Data_PoP\mindMonitor_2023-03-02--19-09-44_b_1.csv
Creating RawArray with float64 data, n_channels=4, n_times=9855
    Range : 0 ... 9854 =      0.000 ...    38.492 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=9855
    Range : 0 ... 9854 =      0.000 ...    38.492 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 100.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transiti

C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:24: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_before.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:27: 

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1691 samples (6.605 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 100.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowe

C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:24: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_before.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:27: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_after.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel

Creating RawArray with float64 data, n_channels=4, n_times=45097
    Range : 0 ... 45096 =      0.000 ...   176.156 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=45097
    Range : 0 ... 45096 =      0.000 ...   176.156 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 100.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition b

...

C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:24: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_before.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1691 samples (6.605 sec)

Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
C:\Users\yehuda\pain_of_payment\Data_PoP\mindMonitor_2023-03-30--13-58-34_b_54.csv


C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:27: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_after.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
C:\Users\yehuda\AppData\Local\anaconda3\envs\my_mne_1.2_env\lib\site-packages\mne\time_frequency\psd.py:56: RuntimeWarning: Mean of empty slice
  spect = np.nanmean(spect, axis=-1)
C:\Users\yehuda\AppData\Local\anaconda3\envs\my_mne_1.2_env\lib\site-packages\mne\time_frequency\psd.py:56: RuntimeWarning: Mean of empty slice
  spect = np.nanmean(spect, axis=-1)
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:39: 

Creating RawArray with float64 data, n_channels=4, n_times=87941
    Range : 0 ... 87940 =      0.000 ...   343.516 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=87941
    Range : 0 ... 87940 =      0.000 ...   343.516 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 100.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition b

C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:24: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_before.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
C:\Users\yehuda\AppData\Local\Temp\ipykernel_3336\3752101634.py:27: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  raw_after.notch_filter(freqs=50, picks = 'eeg')
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel

In [ ]:
ch_names = ['RAW_AF7', 'RAW_AF8', 'ToB', 'TimeStamp']
ch_types = ['misc']+['eeg']*2+['misc']
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
#info.set_montage('standard_1020') #set the electrodes' locations
PSDs_df_before_theta = pd.DataFrame()
PSDs_df_after_theta = pd.DataFrame()
for file in files_eeg:
    print(file)
    data = pd.read_csv(file) #BEN make sure you need to transpose
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp']).map(pd.Timestamp.timestamp)
    data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
    data = data[['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']]
    tob = data['ToB'][0] - data['TimeStamp'][0]
    t_after = tob+5
    t_before = tob-5
    data = data.transpose()
    raw_before = mne.io.RawArray(data, info)
    raw_before.crop(tmin = t_before, tmax = tob)
    raw_after = mne.io.RawArray(data, info)
    raw_after.crop(tmin = tob, tmax = t_after)
    raw_before.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_before.notch_filter(freqs=50, picks = 'eeg')
    #raw_before.plot()
    raw_after.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_after.notch_filter(freqs=50, picks = 'eeg')
    #raw_after.plot()
    psd_before = raw_before.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 4, fmax = 8)
    psd_df_before = psd_before.to_data_frame()
    psd_after = raw_after.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 4, fmax = 8)
    psd_df_after = psd_after.to_data_frame()
    #avergaed_df = psd_df[["freq","RAW_AF7","RAW_AF8"]].mean()
    #avergaed_df=avergaed_df.reset_index().T
    #avergaed_df.columns = avergaed_df.iloc[0]
    #avergaed_df.drop(index=avergaed_df.index[0], axis=0, inplace=True)
    psd_df_before['subject'] = file[-6:-3]
    psd_df_after['subject'] = file[-6:-3]
    PSDs_df_before_theta = PSDs_df_before_theta.append(psd_df_before, ignore_index = True)
    PSDs_df_after_theta = PSDs_df_after_theta.append(psd_df_after, ignore_index = True)

    PSDs_df_after_theta.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\theta_after.csv")
    PSDs_df_before_theta.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\theta_before.csv")

In [ ]:
ch_names = ['RAW_AF7', 'RAW_AF8', 'ToB', 'TimeStamp']
ch_types = ['misc']+['eeg']*2+['misc']
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
#info.set_montage('standard_1020') #set the electrodes' locations
PSDs_df_before_alpha = pd.DataFrame()
PSDs_df_after_alpha = pd.DataFrame()
for file in files_eeg:
    print(file)
    data = pd.read_csv(file) #BEN make sure you need to transpose
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp']).map(pd.Timestamp.timestamp)
    data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
    data = data[['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']]
    tob = data['ToB'][0] - data['TimeStamp'][0]
    t_after = tob+5
    t_before = tob-5
    data = data.transpose()
    raw_before = mne.io.RawArray(data, info)
    raw_before.crop(tmin = t_before, tmax = tob)
    raw_after = mne.io.RawArray(data, info)
    raw_after.crop(tmin = tob, tmax = t_after)
    raw_before.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_before.notch_filter(freqs=50, picks = 'eeg')
    #raw_before.plot()
    raw_after.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_after.notch_filter(freqs=50, picks = 'eeg')
    #raw_after.plot()
    psd_before = raw_before.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 8, fmax = 12)
    psd_df_before = psd_before.to_data_frame()
    psd_after = raw_after.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 8, fmax = 12)
    psd_df_after = psd_after.to_data_frame()
    #avergaed_df = psd_df[["freq","RAW_AF7","RAW_AF8"]].mean()
    #avergaed_df=avergaed_df.reset_index().T
    #avergaed_df.columns = avergaed_df.iloc[0]
    #avergaed_df.drop(index=avergaed_df.index[0], axis=0, inplace=True)
    psd_df_before['subject'] = file[-6:-3]
    psd_df_after['subject'] = file[-6:-3]
    PSDs_df_before_alpha = PSDs_df_before_alpha.append(psd_df_before, ignore_index = True)
    PSDs_df_after_alpha = PSDs_df_after_alpha.append(psd_df_after, ignore_index = True)

    PSDs_df_after_alpha.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\alpha_after.csv")
    PSDs_df_before_alpha.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\alpha_before.csv")

In [ ]:
ch_names = ['RAW_AF7', 'RAW_AF8', 'ToB', 'TimeStamp']
ch_types = ['misc']+['eeg']*2+['misc']
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
#info.set_montage('standard_1020') #set the electrodes' locations
PSDs_df_before_beta = pd.DataFrame()
PSDs_df_after_beta = pd.DataFrame()
for file in files_eeg:
    print(file)
    data = pd.read_csv(file) #BEN make sure you need to transpose
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp']).map(pd.Timestamp.timestamp)
    data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
    data = data[['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']]
    tob = data['ToB'][0] - data['TimeStamp'][0]
    t_after = tob+5
    t_before = tob-5
    data = data.transpose()
    raw_before = mne.io.RawArray(data, info)
    raw_before.crop(tmin = t_before, tmax = tob)
    raw_after = mne.io.RawArray(data, info)
    raw_after.crop(tmin = tob, tmax = t_after)
    raw_before.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_before.notch_filter(freqs=50, picks = 'eeg')
    #raw_before.plot()
    raw_after.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_after.notch_filter(freqs=50, picks = 'eeg')
    #raw_after.plot()
    psd_before = raw_before.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 12, fmax = 30)
    psd_df_before = psd_before.to_data_frame()
    psd_after = raw_after.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 12, fmax = 30)
    psd_df_after = psd_after.to_data_frame()
    #avergaed_df = psd_df[["freq","RAW_AF7","RAW_AF8"]].mean()
    #avergaed_df=avergaed_df.reset_index().T
    #avergaed_df.columns = avergaed_df.iloc[0]
    #avergaed_df.drop(index=avergaed_df.index[0], axis=0, inplace=True)
    psd_df_before['subject'] = file[-6:-3]
    psd_df_after['subject'] = file[-6:-3]
    PSDs_df_before_beta = PSDs_df_before_beta.append(psd_df_before, ignore_index = True)
    PSDs_df_after_beta = PSDs_df_after_beta.append(psd_df_after, ignore_index = True)

    PSDs_df_after_beta.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\beta_after.csv")
    PSDs_df_before_beta.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\beta_before.csv")

In [ ]:
ch_names = ['RAW_AF7', 'RAW_AF8', 'ToB', 'TimeStamp']
ch_types = ['misc']+['eeg']*2+['misc']
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
#info.set_montage('standard_1020') #set the electrodes' locations
PSDs_df_before_gamma = pd.DataFrame()
PSDs_df_after_gamma = pd.DataFrame()
for file in files_eeg:
    print(file)
    data = pd.read_csv(file) #BEN make sure you need to transpose
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp']).map(pd.Timestamp.timestamp)
    data['ToB'][0] = pd.to_datetime(data['ToB'][0]).timestamp()
    data = data[['TimeStamp', 'RAW_AF7', 'RAW_AF8', 'ToB']]
    tob = data['ToB'][0] - data['TimeStamp'][0]
    t_after = tob+5
    t_before = tob-5
    data = data.transpose()
    raw_before = mne.io.RawArray(data, info)
    raw_before.crop(tmin = t_before, tmax = tob)
    raw_after = mne.io.RawArray(data, info)
    raw_after.crop(tmin = tob, tmax = t_after)
    raw_before.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_before.notch_filter(freqs=50, picks = 'eeg')
    #raw_before.plot()
    raw_after.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
    raw_after.notch_filter(freqs=50, picks = 'eeg')
    #raw_after.plot()
    psd_before = raw_before.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 31, fmax = 45)
    psd_df_before = psd_before.to_data_frame()
    psd_after = raw_after.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'], fmin = 31, fmax = 45)
    psd_df_after = psd_after.to_data_frame()
    #avergaed_df = psd_df[["freq","RAW_AF7","RAW_AF8"]].mean()
    #avergaed_df=avergaed_df.reset_index().T
    #avergaed_df.columns = avergaed_df.iloc[0]
    #avergaed_df.drop(index=avergaed_df.index[0], axis=0, inplace=True)
    psd_df_before['subject'] = file[-6:-3]
    psd_df_after['subject'] = file[-6:-3]
    PSDs_df_before_gamma = PSDs_df_before_gamma.append(psd_df_before, ignore_index = True)
    PSDs_df_after_gamma = PSDs_df_after_gamma.append(psd_df_after, ignore_index = True)

PSDs_df_after_gamma.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\gamma_after.csv")
PSDs_df_before_gamma.to_csv(r"C:\Users\Enbal\Documents\GitHub\Ben_analysis\gamma_before.csv")

In [ ]:
#gamma
compute_freq(files_eeg, 31, 45)

In [ ]:
PSDs_df_after.plot()

In [ ]:
#This is for a single file! FAA with different conditions - OLD
filepath = r"C:\Users\Enbal\Dropbox\LabDropbox\Inbal\1st EEG\EEGResults_converted\201.csv"
data = pd.read_csv(filepath).transpose()
ch_names = ['F7', 'Fp1', 'Fpz', 'Fp2', 'F8', 'Fz', 'Cz', 'Pz', 'accel1', 'accel2', 'accel3', 'triggers', 'timestamp']
ch_types = ['eeg']*8+['misc']*5
sfreq = 500
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
info.set_montage('standard_1020') #set the electrodes' locations
raw = mne.io.RawArray(data, info)
raw = mne.io.RawArray(data, info)
raw.filter(l_freq = 1, h_freq = 100, picks = 'eeg', method = 'iir') #check for the use of iir_params - these can be specified to be more accute. ALSO, check if I want FIR or IIR. 
raw.notch_filter(freqs=50, picks = 'eeg')
#raw.plot()
#print(raw.info)

#to divide the events into two arrays - for bdm1 and bdm2:
events = mne.find_events(raw, stim_channel='triggers')
locations = np.where(events == 1) #take only the triggers locations of the trigger=1
locations = locations[0] #leave only a list of the locations of the releavnt trigger
locations = np.subtract(locations, 1) #take the location of the triggers before the 1 trigger, as this is when the stimulus ends
a = np.split(events, locations) #split the events array into three arrays based on the releavnt event locations
events_bdm1 = a[1] #assign the right arrays to each of the releavnt parts :)
events_bdm2 = a[2]

list_bdm1_high_201 = bdm1_high_df_list_for_EEG[0]['ItemIndex'].values.tolist() #make the triggers from the behavioral experiment a list
list_bdm1_low_201 = bdm1_low_df_list_for_EEG[0]['ItemIndex'].values.tolist()
list_bdm2_high_201 = bdm2_high_df_list_for_EEG[0]['ItemIndex'].values.tolist()
list_bdm2_low_201 = bdm2_low_df_list_for_EEG[0]['ItemIndex'].values.tolist()
events_bdm1_high_201 = events_bdm1[np.in1d(events_bdm1[:,2], list_bdm1_high_201)] #take only the events of the relevant condition
events_bdm1_low_201 = events_bdm1[np.in1d(events_bdm1[:,2], list_bdm1_low_201)]
events_bdm2_high_201 = events_bdm2[np.in1d(events_bdm2[:,2], list_bdm2_high_201)]
events_bdm2_low_201 = events_bdm2[np.in1d(events_bdm2[:,2], list_bdm2_low_201)]
#reject_criteria = dict(eeg=150e-6)       # 150 µV
epochs_bdm1_high_201 = mne.Epochs(raw, events_bdm1_high_201, tmin=-6, tmax=0.5,
                    #reject=reject_criteria,
                    preload=True) #understand better the reject criteria!! if I use it now, it drops all epochs! also, I need to understand how to epoch better. Right now I epoch based on all events, but I need to decide how exectly to do it...
epochs_bdm1_low_201 = mne.Epochs(raw, events_bdm1_low_201, tmin=-6, tmax=0.5,
#reject=reject_criteria,
preload=True) #understand better the reject criteria!! if I use it now, it drops all epochs! also, I need to understand how to epoch better. Right now I epoch based on all events, but I need to decide how exectly to do it...
epochs_bdm2_high_201 = mne.Epochs(raw, events_bdm2_high_201, tmin=-6, tmax=0.5,
                    #reject=reject_criteria,
                    preload=True) #understand better the reject criteria!! if I use it now, it drops all epochs! also, I need to understand how to epoch better. Right now I epoch based on all events, but I need to decide how exectly to do it...
epochs_bdm2_low_201 = mne.Epochs(raw, events_bdm2_low_201, tmin=-6, tmax=0.5,
#reject=reject_criteria,
preload=True) #understand better the reject criteria!! if I use it now, it drops all epochs! also, I need to understand how to epoch better. Right now I epoch based on all events, but I need to decide how exectly to do it...

#fig = mne.viz.plot_events(events, event_id=event_dict, sfreq=raw.info['sfreq'], first_samp=raw.first_samp)
mne.viz.plot_events(events_bdm2_high_201, sfreq=raw.info['sfreq'], first_samp=raw.first_samp)

psd_bdm1_high_201 = epochs_bdm1_high_201.compute_psd(method = 'welch', fmin = 8, fmax = 13, picks = ['F7', 'F8'])
df_psd_bdm1_high_201 = psd_bdm1_high_201.to_data_frame() 
df2 = df_psd_bdm1_high_201[["freq","F7","F8"]].mean()
df3=df2.reset_index().T
df3.columns = df3.iloc[0]
df3.drop(index=df3.index[0], axis=0, inplace=True)
df3['F7_log'] = math.log(df3['F7'])
df3['F8_log'] = math.log(df3['F8'])
df3['alpha_asymmetry'] = df3['F7_log']-df3['F8_log']

epochs_bdm1_high.compute_psd().plot_topomap()
epochs_bdm1_low.compute_psd().plot_topomap()

In [ ]:
#find the triggers - old..
#events = mne.find_events(raw, stim_channel='triggers')
#print(events[:10])  # show the first 10
#rows=np.where(events[:,2]>105) #1-105 are the triggers signifting the appearance of the slider, and hence the response. 
#eventsNew=(events[rows])